<a href="https://colab.research.google.com/github/yasminreich/comp_bio_targil3/blob/population/Population.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import Individual
import Mutation
import numpy as np
import math, random
from sklearn.model_selection import train_test_split


In [2]:
class Population:
    def __init__(self, train_data, train_labels, size, F1Threshold):
        self.size = size
        self.population = []
        self.numOfLayers = 3
        self.layersSizes = [4, 8, 32]
        self.train_data = train_data
        self.train_labels = train_labels
        self.F1Threshold = F1Threshold
        
        self.__createInitialPop()

    
    def __createInitialPop(self):
        inputSize = self.train_data.shape[2]
        for i in range(self.size):
            nn = Individual.NN(inputSize, self.F1Threshold, self.numOfLayers, self.layersSizes, self.train_data, self.train_labels)
            self.population.append(nn)
    
    def runTrain(self):
        for p in self.population:
            p.train()
            p.calculateFitness()


    def dispachBestPeople(self, bestPeople):

        newPop = []
        for person in bestPeople:
            # create new copies of the best individual (5% of the new population)
            percent = person.fitness/10*100
            amount = math.ceil((self.size/100)*percent)
            if amount == 0:
                amount = 1
            for i in range(amount):
                newPop.append(person.deepcopy())
        
        return newPop

    # remove individuals with low fitness
    def __naturalSelection(self, percent):

        # devidor = self.__getDevidor(percentileNum)
        temp = []
        people_to_remove = int(self.size * percent)
        temp = self.population[:self.size - people_to_remove]
        self.population = temp

                

    def nextGen(self, deathThreshold, mutationChance):
        self.runTrain()
        self.population = sorted(self.population, key=lambda p: p.fitness, reverse=True)

        self.bestPerson = self.population[0]
        self.__naturalSelection(deathThreshold)
        
        newPopulation = self.dispachBestPeople(self.population[:5])

        popForCros = []
        # create vector of people for the crossover
        for person in self.population:
            fit = person.fitness*100
            for _ in range(int(fit)):
                popForCros.append(person.deepcopy())
        

        while True:
            # print("in crossover in next gen")
            parent1, parent2 = random.sample(popForCros, 2)
            child1, child2 = Mutation.crossover(parent1, parent2)
            newPopulation.append(child1)
            if len(newPopulation) >= self.size:
                break
            newPopulation.append(child2)
            if len(newPopulation) >= self.size:
                break
        
        self.population = newPopulation

        for person in self.population:
            if random.random() < mutationChance:
                Mutation.mutate_individual(person)





In [3]:
def read_data(file_path):
    samples = []
    labels = []

    with open(file_path, 'r') as file:
        for line in file:
            data = line.strip().split('   ')
            if len(data) >= 2:
                sample = [int(bit) for bit in data[0].strip() if bit.isdigit()]
                label = int(data[1])
                
                samples.append(np.array(sample).reshape(1, len(sample)))
                labels.append(label)

    return np.array(samples), np.array(labels)

In [4]:
data, labels = read_data("nn0.txt")

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

popy = Population(train_data, train_labels, size=60, F1Threshold=0.3)
deathThreshold=0.2
mutationChance=0.6
convergenceMax = 10
convergenceCount = 0
generationCounter = 0
lastBestFit = 0
while convergenceCount < convergenceMax:
    generationCounter += 1
    popy.nextGen(deathThreshold=deathThreshold, mutationChance=mutationChance)
    # print("best person fitness:", float(popy.bestPerson.fitness))
    print("best person accuracy:", float(popy.bestPerson.accuracy))
    if popy.bestPerson.fitness == lastBestFit:
        # break
        convergenceCount += 1
    else:
        convergenceCount = 0
    lastBestFit = popy.bestPerson.fitness

print(popy.bestPerson.test(test_data, test_labels))

best person fitness: 0.4874375
best person accuracy: 0.4874375
best person fitness: 0.585875
best person accuracy: 0.585875
best person fitness: 0.5868125
best person accuracy: 0.5868125
best person fitness: 0.5868125
best person accuracy: 0.5868125
best person fitness: 0.586875
best person accuracy: 0.586875
best person fitness: 0.586875
best person accuracy: 0.586875
best person fitness: 0.5870625
best person accuracy: 0.5870625
best person fitness: 0.6021875
best person accuracy: 0.6021875
best person fitness: 0.6021875
best person accuracy: 0.6021875
best person fitness: 0.6035
best person accuracy: 0.6035
best person fitness: 0.6070625
best person accuracy: 0.6070625
best person fitness: 0.608875
best person accuracy: 0.608875
best person fitness: 0.6268125
best person accuracy: 0.6268125
best person fitness: 0.6279375
best person accuracy: 0.6279375
best person fitness: 0.6356875
best person accuracy: 0.6356875
best person fitness: 0.6389375
best person accuracy: 0.6389375
best p

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/my/jc643vgs3_304rv4scm_170h0000gn/T/ipykernel_69797/3206280349.py", line 14, in <cell line: 12>
    popy.nextGen(deathThreshold=deathThreshold, mutationChance=mutationChance)
  File "/var/folders/my/jc643vgs3_304rv4scm_170h0000gn/T/ipykernel_69797/1359459363.py", line 52, in nextGen
    self.runTrain()
  File "/var/folders/my/jc643vgs3_304rv4scm_170h0000gn/T/ipykernel_69797/1359459363.py", line 22, in runTrain
    p.train()
  File "/Users/chenbistra/Documents/repos/comp_bio_targil3/Individual.py", line -1, in train
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core